# Aplicação RAG

![Simple RAG](../../images/simple_rag.png)

Neste notebook, vamos configurar uma aplicação RAG simples que usaremos para aprender mais sobre o LangSmith.

RAG (Retrieval Augmented Generation) é uma técnica popular para fornecer aos LLMs documentos relevantes que permitirão que respondam melhor às perguntas dos usuários.

No nosso caso, vamos indexar documentação do LangSmith!

O LangSmith facilita o rastreamento de qualquer aplicação LLM, sem necessidade de LangChain!

### Configuração

Certifique-se de definir suas variáveis de ambiente, incluindo sua chave de API do Google.

In [ ]:
# Você pode defini-las diretamente!
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Aplicação RAG Simples

In [ ]:
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat, get_gemini_model_name

from langsmith import traceable
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "google"
MODEL_NAME = "gemini-1.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Retorna documentos buscados de um armazenamento vetorial baseado na pergunta do usuário
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Chama `call_gemini` para gerar uma resposta do modelo após formatar entradas
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Contexto: {formatted_docs} \n\n Pergunta: {question}"
        }
    ]
    return call_gemini(messages)

"""
call_gemini
- Retorna a saída de conclusão de chat do Google Gemini
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_gemini(messages: List[dict], temperature: float = 0.0) -> str:
    return call_gemini_chat(MODEL_NAME, messages, temperature)

"""
langsmith_rag
- Chama `retrieve_documents` para buscar documentos
- Chama `generate_response` para gerar uma resposta baseada nos documentos buscados
- Retorna a resposta do modelo
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

Isso deve levar um pouco menos de um minuto. Estamos indexando e armazenando documentação do LangSmith em um banco de dados vetorial SKLearn.

In [ ]:
question = "Para que é usado o LangSmith?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)

### Vamos dar uma olhada no LangSmith!